### Import dependenties

In [1]:
import sys
from pathlib import Path
import sys
from PyQt6.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QComboBox, QPushButton, QFileDialog, QTextEdit, QLabel
from PyQt6.QtGui import QIcon, QPalette, QColor
from PyQt6.QtCore import Qt
from PyQt6.QtCore import QDateTime

notebook_directory = Path().resolve().parent.parent
sys.path.append(str(notebook_directory))

print(notebook_directory)
from bp_single_file import *

C:\Users\Jonathan\Documents\GitHub\building.py


#### - select in app

In [ ]:
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("BuildingPy Convert")
        self.setGeometry(100, 100, 800, 400)

        self.initUI()

    def initUI(self):
        # Set a modern gray background color
        palette = QPalette()
        palette.setColor(QPalette.ColorRole.Window, QColor(45, 45, 45))
        palette.setColor(QPalette.ColorRole.WindowText, QColor(240, 240, 240))
        palette.setColor(QPalette.ColorRole.Button, QColor(75, 75, 75))
        palette.setColor(QPalette.ColorRole.ButtonText, QColor(240, 240, 240))
        palette.setColor(QPalette.ColorRole.Highlight, QColor(60, 150, 200))
        palette.setColor(QPalette.ColorRole.HighlightedText, QColor(240, 240, 240))
        self.setPalette(palette)

        # Create a central widget
        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        # Create a vertical layout for the central widget
        layout = QVBoxLayout()
        central_widget.setLayout(layout)

        # Create a horizontal layout for the sections
        section_layout = QHBoxLayout()

        # Left Section
        left_section = QWidget()
        left_layout = QVBoxLayout()
        left_section.setLayout(left_layout)

        left_label = QLabel("From:")
        left_layout.addWidget(left_label)

        left_dropdown = QComboBox()
        left_dropdown.addItems(["Scia"])
        left_dropdown.setStyleSheet("font-size: 16px;")
        left_layout.addWidget(left_dropdown)
        left_dropdown.setObjectName("left_dropdown")

        section_layout.addWidget(left_section)

        # Transfer Icon
        transfer_icon = QLabel()
        transfer_icon.setPixmap(QIcon("transfer.ico").pixmap(32, 32))
        section_layout.addWidget(transfer_icon)

        # Right Section
        right_section = QWidget()
        right_layout = QVBoxLayout()
        right_section.setLayout(right_layout)

        right_label = QLabel("To:")
        right_layout.addWidget(right_label)

        right_dropdown = QComboBox()
        right_dropdown.addItems(["Xfem4u", "Speckle"])
        right_dropdown.setStyleSheet("font-size: 16px;")
        right_layout.addWidget(right_dropdown)

        section_layout.addWidget(right_section)

        layout.addLayout(section_layout)

        # Button to select XML file
        convert_button = QPushButton("Convert")
        convert_button.clicked.connect(self.test)
        convert_button.setStyleSheet("font-size: 16px;")
        layout.addWidget(convert_button)

        # Output Text Area
        self.output_text = QTextEdit()
        # Set the background color of the output text area to gray
        self.output_text.setStyleSheet("background-color: lightgray;")
        layout.addWidget(self.output_text)

    def test(self):
        print(1)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.setWindowIcon(QIcon("icon.ico"))
    window.show()
    sys.exit(app.exec())


In [ ]:
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Scia2XFEM4U converter")
        button = QPushButton("Select XML File")
        button.clicked.connect(self.select_xml_file)
        self.setCentralWidget(button)

    def select_xml_file(self):
        filename, _ = QFileDialog.getOpenFileName(self, "Select XML File", filter="XML Files (*.xml)")
        if filename:
            print("Selected XML file:", filename)
        else:
            print("No XML file selected")


app = QApplication(sys.argv)

window = MainWindow()
window.show()

app.exec()

#


### Convert Scia to XFEM4U

In [3]:
#convert data here using the single file

#.xml file select from scia and read, convert them to buldingpy objects and then convert them to struct4u xml file.

filepath = f"{notebook_directory}\\temp\\Scia\\Examples buildingpy\\_1.xml"

project = BuildingPy("TempCommit", "0")

LoadXML(filepath, project)
# print(project.objects)
# project.toSpeckle("c6e11e74cb")

### test
#### test


### Write to the .xml (save in documents)

In [4]:
base_file_name = "Struct4U_XFEM4U_export.xml"
documents_dir = os.path.expanduser("~/Documents")

file_name = base_file_name
counter = 1

while os.path.exists(os.path.join(documents_dir, file_name)):
    file_name = f"Struct4U_XFEM4U_export_{counter}.xml"
    counter += 1

path = os.path.join(documents_dir, file_name).replace("\\", "/")

# write to .xml struct4u file
xmlS4U = xmlXFEM4U() # Create XML object with standard values
xmlS4U.addBeamsPlates(project.objects) #Add Beams, Profiles, Plates, Beamgroups, Nodes
xmlS4U.addProject("Examples of building.py")
xmlS4U.addPanels(project.objects) #add Load Panels
xmlS4U.addSurfaceLoad(project.objects)
xmlS4U.addLoadCasesCombinations()
xmlS4U.XML()
XMLString = xmlS4U.xmlstr

file = open(path, "w")
a = file.write(XMLString)
file.close()

print(f"Custom XML file '{file_name}' has been created in the Documents directory.")

Custom XML file 'Struct4U_XFEM4U_export_4.xml' has been created in the Documents directory.


### Save the path in the .ini file

In [5]:
import os
import configparser

ini_file_path = os.path.join(os.getenv('LOCALAPPDATA'), 'Struct4u', 'DirectCommands_XFEM4U.ini')
config = configparser.ConfigParser()

config['Struct4u'] = {
    'Import_XML': f'{path}'
}

if os.path.exists(ini_file_path):
    with open(ini_file_path, 'w') as config_file:
        config.write(config_file)
else:
    os.makedirs(os.path.dirname(ini_file_path), exist_ok=True)
    with open(ini_file_path, 'w') as config_file:
        config.write(config_file)

print(f".ini file '{ini_file_path}' has been updated/created.")

.ini file 'C:\Users\Jonathan\AppData\Local\Struct4u\DirectCommands_XFEM4U.ini' has been updated/created.


### open XFEM4U program

In [1]:
import subprocess
subprocess.call("C:/Struct4u/XFEM4U/wframe3d.exe", shell=True)